# Straight Wire Magnetic Field Demo

In [1]:
import numpy as np
from em_app.sources import StraightWire
from em_app.solvers import calculate_b_field
from mtflib import mtf

mtf.initialize_mtf(max_order=6, max_dimension=4)

## 1. Numerical B-Field Calculation

In [2]:
start_point = [0, 0, -1]
end_point = [0, 0, 1]
current = 1.0
num_segments = 200

wire = StraightWire(current, start_point, end_point, num_segments)
observation_point = np.array([[0.1, 0, 0]])
b_field = calculate_b_field(wire, observation_point)
B_numerical = b_field._b_vectors_mtf[0].to_numpy_array()

print(f"Computed B-field: {B_numerical}")

## 2. Analytical B-Field Calculation

In [3]:
mu_0 = 4 * np.pi * 1e-7
a = 0.1
z1 = -1.0
z2 = 1.0
z = 0

cos_theta_1 = (z - z1) / np.sqrt((z - z1) ** 2 + a**2)
cos_theta_2 = (z - z2) / np.sqrt((z - z2) ** 2 + a**2)

B_phi_mag = (mu_0 * current / (4 * np.pi * a)) * (cos_theta_1 - cos_theta_2)

print(f"Analytical B-field (magnitude): {B_phi_mag}")